# Notebook 02 - Tratamento silver
Para realizar as análises que desejamos, precisamos garantir que os dados estejam prontos e que respondam as perguntas objetivo. Para cada pergunta, temos dados que precisamos garantir que estejam confiáveis, eles estão listados abaixo:
**Perguntas Objetivo:**

Perguntas relacionadas aos registros em todo o estado de São Paulo:

- Quais as 5 categorias de crime mais registrados no Estado? (Criação da coluna `categoria`)
- Esses 5 permaneceram os mesmos em 2022, 2023 e 2024? (Criação de um gráfico, colunas `categoria` e `ano`)
- Houve sazonalidade? Com naturezas mais registradas ou menos registradas em determinadas épocas do ano? Ou eventos de aumento ou redução drásticas de determinados registros? (Criação de um gráfico, colunas `categoria` , `ano`, e `mes`)
- Qual a distribuição geográfica das categorias de crime dentro do estado de São Paulo? (Mapa de São Paulo com pontos, colunas `latitude`e `longitude`)

Perguntas relacionadas à distribuição dos tipos ou categorias de crime nos diferentes municípios do estado:

- Qual o percentual de registro de cada tipo de crime nos anos de 2022, 2023 e 2024 em cada município de São Paulo? (Detalhamento do Mapa de São Paulo, colunas `municipio`, `codigo_municipio` e `tipo_crime`)
- Qual a distribuição geográfica das categorias de crime dentro de cada município? (Detalhamento do Mapa de São Paulo, colunas `municipio`, `codigo_municipio`, `latitude`, `longitude`e `categorias`)
- Qual a taxa de registro a cada 100 mil habitantes das 3 maiores categorias de crime de cada município? (Cálculo da coluna `taxa_registros` na etapa Gold, com as colunas `categoria` , `municipio`, `pop_censo_2022`  e `pop_estimada_2024` .

Com isso em mente, vamos seguir para o tratamento dos dados.

## 1. Setup Inicial
Baixando os pacotes necessários, importando as bibliotecas (da melhor forma possível para o Databricks), e ajustando as configurações para a melhor forma.

In [0]:
# baixando o unidecode
%pip install unidecode

In [0]:
# importando o básico necessário do pyspark, separado se não o Databricks "esquece"
from pyspark.sql.functions import col, when, count, round, countDistinct, isnull
from pyspark.sql.types import StringType

In [0]:
# importando as outras bibliotecas pyspark separadas também, para evitar esquecimentos do Databricks
from pyspark.sql.functions import (
    concat_ws, lpad, substring, expr, monotonically_increasing_id,
    lower, upper, trim, regexp_replace, create_map, lit, sum
)
from pyspark.sql import functions as F
from pyspark.sql.functions import monotonically_increasing_id

In [0]:
import unicodedata
from unidecode import unidecode

In [0]:
# importando as bibliotecas de análise e visualização que vamos utilizar (que não são pyspark)
import re
from itertools import chain

%matplotlib inline
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [0]:
pd.set_option("display.max_rows", None)  # mostra todas as linhas

In [0]:
pd.set_option("display.max_colwidth", None)  # exibe nomes completos sem cortar

In [0]:
pd.set_option("display.max_columns", None)  # mostra todas as colunas, mesmo se tiver muitas

In [0]:
# Diagnóstico: o cluster sabe o que é "when"?
try:
    print("Tipo de 'when':", type(when))
except NameError:
    print("O cluster está fingindo que não conhece o 'when'")

Depois de uma série de problemas com as importações, consegui chegar nesse formato, que funciona no Databricks.

Agora vamos importar os dados que vamos precisar.

In [0]:
# caminhos para os dados de criminalidade
caminho_amostra = "/FileStore/mvp_criminalidade_sp/bronze/amostra_representativa.parquet"
caminho_bronze_full = "/FileStore/mvp_criminalidade_sp/bronze/full_bronze.parquet"
caminho_ibge = "dbfs:/FileStore/mvp_criminalidade_sp/bronze/ibge_sp.csv"

In [0]:
# leitura da amostra representativa dos dados de criminalidade
df_amostra = spark.read.parquet(caminho_amostra)

# visualização inicial
df_amostra.printSchema()

In [0]:
# leitura do parquet bronze completo de criminalidade
df_bronze = spark.read.parquet(caminho_bronze_full)
df_bronze.printSchema()

In [0]:
# leitura dos dados do ibge para o estado de sao paulo - que são complementares
df_ibge = spark.read.option("header", True).option("inferSchema", True).csv(caminho_ibge)

df_ibge.printSchema()

## 2. Análise Exploratória dos Dados
Vamos verificar diversos aspectos dos dados para compreender quais etapas de limpeza serão necessárias.

#### Primeiro vamos selecionar os atributos e colunas que vamos necessitar na análise
Vamos checar o dicionário de dados para entender quais vamos precisar para construir a análise:

|Atributos|Descrição|
|---|---|
|ANO\_BO|Ano do Boletim|
|ANO\_ESTATISTICA|Ano em que a ocorrência foi inserida na estatística oficial |
|BAIRRO|Bairro da Ocorrência|
|CIDADE|Cidade de Registro|
|DATA\_COMUNICACAO|Data da Comunicação|
|DATA\_OCORRENCIA\_BO|Data da Ocorrência|
|DESC\_PERIODO|Período da Ocorrência|
|DESCR\_CONDUTA|Tipo de local ou circunstancia que qualifica a ocorrencia|
|DESCR\_TIPOLOCAL|Descreve grupo de tipos de locais onde se deu o fato|
|HORA\_OCORRENCIA\_BO|Hora da Ocorrência|
|LATITUDE|Latitude da Ocorrência|
|LOGRADOURO|Logradouro dos fatos|
|LONGITUDE|Longitude da Ocorrência|
|MÊS ESTATISTICA|Mês em que a ocorrência foi inserida na estatística oficial |
|NATUREZA\_APURADA|Natureza de Publicação|
|NOME\_DELEGACIA|Delegacia responsável pelo registro|
|NOME\_DELEGACIA\_CIRCUNSCRIÇÃO|Delegacia de Circunscrição|
|NOME\_DEPARTAMENTO|Departamento responsável pelo registro|
|NOME\_DEPARTAMENTO\_CIRCUNSCRIÇÃO|Departamento de Circunscrição|
|NOME\_MUNICIPIO\_CIRCUNSCRIÇÃO|Município da Delegacia de Circunscrição|
|NOME\_SECCIONAL|Delegacia Seccional responsável pelo registro|
|NOME\_SECCIONAL\_CIRCUNSCRIÇÃO|Seccional de Circunscrição|
|NUM\_BO|Número do Boletim|
|NUMERO\_LOGRADOURO|Numero do Logradouro dos fatos|
|RUBRICA|Natureza juridica da ocorrencia|


Para construir o dashboard, vamos precisar das informações:
- Cidade em que o crime foi cometido: presente na coluna `NOME_MUNICIPIO_CIRCUNSCRIÇÃO`
- Ano e mês em que o crime foi incluído na estatística: presente nas colunas `ANO_ESTATISTICA` e `MÊS ESTATISTICA`
- O tipo de crime: podemos utilizar o campo `NATUREZA_APURADA` ou `RUBRICA`, **precisamos avaliar**
- O local onde os crimes foram cometidos (para o mapa de ocorrências das cidades e municípios): para isso precisamos de `LATITUDE` e `LONGITUDE`. Dependendo da quantidade de registros sem esses dados, não vamos gerar essa análise para não perdermos os registros, por isso **precisamos avaliar**.


Embora o detalhamento por bairro oferecesse um grau maior de granularidade, a inconsistência nos dados de origem comprometeu a confiabilidade dessa análise. Assim, optamos por focar no nível municipal neste MVP, propondo o refino e a padronização dos dados de bairro como trabalho futuro.

### 2.1 Analisando a diferença entre Natureza Apurada e Rubrica
Para decidirmos qual campo vamos usar, vamos checar a contagem de valore para cada coluna.

In [0]:
from pyspark.sql.functions import count
# contagem de NATUREZA_APURADA
print("Contagem por NATUREZA_APURADA")
df_amostra.groupBy("NATUREZA_APURADA") \
    .agg(count("*").alias("quantidade")) \
    .orderBy("quantidade", ascending=False) \
    .show(truncate=False)

# contagem de RUBRICA
print("Contagem por RUBRICA")
df_amostra.groupBy("RUBRICA") \
    .agg(count("*").alias("quantidade")) \
    .orderBy("quantidade", ascending=False) \
    .show(truncate=False)

Observando os valores das duas colunas, vamos seguir com `NATUREZA APURADA` como principal, já que possui as categorias bem definidas e provavelmente foi a informação mais atualizada e publicada, por ser chamada de 'Natureza da Publicação'.

### 2.2 Analisando os nulos em Latitude e Longitude
Vamos checar os nulos no banco de dados inteiro, assim teremos uma visão mais realista.

In [0]:
total_registros = df_bronze.count()

df_bronze.select([
    count(when(col("LATITUDE").isNull(), True)).alias("LATITUDE_NULOS"),
    round((count(when(col("LATITUDE").isNull(), True)) / total_registros) * 100, 2).alias("LATITUDE_PERC_NULO"),

    count(when(col("LONGITUDE").isNull(), True)).alias("LONGITUDE_NULOS"),
    round((count(when(col("LONGITUDE").isNull(), True)) / total_registros) * 100, 2).alias("LONGITUDE_PERC_NULO"),

    count("*").alias("TOTAL_REGISTROS")
]).show()

In [0]:
# criando dfs filtradas
df_total = df_bronze.filter((col("NATUREZA_APURADA").isNotNull()) & (col("NOME_MUNICIPIO_CIRCUNSCRIÇÃO").isNotNull()))
df_geo = df_total.filter((col("LATITUDE").isNotNull()) & (col("LONGITUDE").isNotNull()))
df_nogeo = df_total.filter((col("LATITUDE").isNull()) | (col("LONGITUDE").isNull()))

# totais
total_total = df_total.count()
total_geo = df_geo.count()
total_nogeo = df_nogeo.count()

# distribuicao por municipio
dist_total_municipio = df_total.groupBy("NOME_MUNICIPIO_CIRCUNSCRIÇÃO") \
    .agg(count("*").alias("qtd_total")) \
    .withColumn("perc_total", round(col("qtd_total") / total_total * 100, 2))

dist_geo_municipio = df_geo.groupBy("NOME_MUNICIPIO_CIRCUNSCRIÇÃO") \
    .agg(count("*").alias("qtd_geo")) \
    .withColumn("perc_geo", round(col("qtd_geo") / total_geo * 100, 2))

dist_nogeo_municipio = df_nogeo.groupBy("NOME_MUNICIPIO_CIRCUNSCRIÇÃO") \
    .agg(count("*").alias("qtd_nogeo")) \
    .withColumn("perc_nogeo", round(col("qtd_nogeo") / total_nogeo * 100, 2))

# juntar comparativos
comparativo_municipio = dist_total_municipio \
    .join(dist_geo_municipio, on="NOME_MUNICIPIO_CIRCUNSCRIÇÃO", how="left") \
    .join(dist_nogeo_municipio, on="NOME_MUNICIPIO_CIRCUNSCRIÇÃO", how="left") \
    .fillna(0) \
    .orderBy(col("perc_total").desc())

print("Comparativo de MUNICÍPIOS - Total vs Geo vs Sem Geo")
comparativo_municipio.show(truncate=False)


In [0]:
# criando dfs filtradas
df_total = df_bronze.filter((col("NATUREZA_APURADA").isNotNull()) & (col("NATUREZA_APURADA").isNotNull()))
df_geo = df_total.filter((col("LATITUDE").isNotNull()) & (col("LONGITUDE").isNotNull()))
df_nogeo = df_total.filter((col("LATITUDE").isNull()) | (col("LONGITUDE").isNull()))

# totais
total_total = df_total.count()
total_geo = df_geo.count()
total_nogeo = df_nogeo.count()

# distribuicao por natureza
dist_total_natureza = df_total.groupBy("NATUREZA_APURADA") \
    .agg(count("*").alias("qtd_total")) \
    .withColumn("perc_total", round(col("qtd_total") / total_total * 100, 2))

dist_geo_natureza = df_geo.groupBy("NATUREZA_APURADA") \
    .agg(count("*").alias("qtd_geo")) \
    .withColumn("perc_geo", round(col("qtd_geo") / total_geo * 100, 2))

dist_nogeo_natureza = df_nogeo.groupBy("NATUREZA_APURADA") \
    .agg(count("*").alias("qtd_nogeo")) \
    .withColumn("perc_nogeo", round(col("qtd_nogeo") / total_nogeo * 100, 2))

# juntar comparativos
comparativo_natureza = dist_total_natureza \
    .join(dist_geo_natureza, on="NATUREZA_APURADA", how="left") \
    .join(dist_nogeo_natureza, on="NATUREZA_APURADA", how="left") \
    .fillna(0) \
    .orderBy(col("perc_total").desc())

print("Comparativo de NATUREZA APURADA - Total vs Geo vs Sem Geo")
comparativo_natureza.show(truncate=False)


Como podemos observar, a quantidade de nulos é relativamente alta (10%) e retirar esses valores pode alterar a representatividade dos dados. Em uma análise mais aprofundada, poderíamos preencher esses dados com base nas colunas `BAIRRO`, `LOGRADOURO` e `NUMERO_LOGRADOURO`.

Por isso vamos plotar os gráficos do dashboard com a `LATITUDE` e `LONGITUDE` existentes e utilizar os dados completos no restante das análises.

### 2.3 Checando as duplicadas
Vamos checar se existem dados duplicados na base, antes de seguir para a limpeza.

#### 2.3.1 Base Criminalidade

In [0]:
print(f"Total de linhas da base de criminalidade: {df_bronze.count()}")
print(f"Total de linhas únicas da base de criminalidade: {df_bronze.dropDuplicates().count()}")

Como a quantidade de nulos é absolutamente muito pequena (apenas 16 em 3,6 milhões) vamos remover esses registros.

#### 2.3.2 Base IBGE

In [0]:
print(f"Total de linhas da base ibge: {df_ibge.count()}")
print(f"Total de linhas únicas da base ibge: {df_ibge.dropDuplicates().count()}")

Como a quantidade de nulos é absolutamente muito pequena (apenas 1 em 660) vamos remover esses registros.

In [0]:
# removendo as duplicidades (mas mantendo um registro)
df_ibge = df_ibge.dropDuplicates()

## 3. Limpeza dos Dados
A seguir vamos realizar as etapas necessárias para deixar os dados prontos para o dashboard e as análises. Primeiro vamos limpar a base principal, a de criminalidade. Por fim vamos para a complementar, a do IBGE.

### 3.1 Padronização do nome das colunas
Vamos deixar todas em letra minúscula, sem acentos e com traço, em _snake case_.

#### 3.1.1 Base Criminalidade

In [0]:
def padronizar_colunas(df):
    novos_nomes = [unidecode(c.lower().strip().replace(" ", "_")) for c in df.columns]
    return df.toDF(*novos_nomes)

df_silver = padronizar_colunas(df_bronze)
df_silver.printSchema()

#### 3.1.2 Base IBGE

In [0]:
# Função para normalizar nomes de colunas
def normalizar_coluna(nome):
    nome = nome.strip().lower()
    nome = unicodedata.normalize("NFKD", nome)
    nome = "".join(c for c in nome if not unicodedata.combining(c))
    nome = nome.replace(" ", "_")
    nome = "".join(c for c in nome if c.isalnum() or c == "_")
    return nome

# Aplicar nos nomes das colunas
df_ibge = df_ibge.toDF(*[normalizar_coluna(c) for c in df_ibge.columns])

# Função para remover "_" final (se houver) dos nomes das colunas
def remover_underline_final(nome):
    return nome[:-1] if nome.endswith("_") else nome

# Aplicar aos nomes já normalizados
df_ibge = df_ibge.toDF(*[remover_underline_final(c) for c in df_ibge.columns])

df_ibge.printSchema()

### 3.2 Seleção das colunas que iremos utilizar

#### 3.2.1 Base Criminalidade
De acordo com as análises que vamos fazer, vamos precisar de determinadas colunas:
- A chave primária `num_bo`: para termos a identificação unitária de cada linha
- Cidade em que o crime foi cometido: presente na coluna `nome_municipio_circunscricao`
- Ano e mês em que o crime foi incluído na estatística: presente nas colunas `ano_estatistica` e `mes_estatistica`
- O tipo de crime: vamos utilizar o campo `natureza_apurada`
- O local onde os crimes foram cometidos (para o mapa de ocorrências das cidades e municípios): para isso precisamos de `latitude` e `longitude`. Mantendo os registros sem dados, mas não incluindo nos gráficos.


Em seguida vamos criar uma lista com apenas as colunas que iremos utilizar.

In [0]:
# selecionando as colunas que iremos utilizar
colunas_relevantes = ['num_bo', 'nome_municipio_circunscricao', 'ano_estatistica', 'mes_estatistica', 
                      'natureza_apurada', 'latitude', 'longitude']

# filtrando as colunas relevantes
df_silver = df_silver.select(*colunas_relevantes)
df_silver.printSchema()

#### 3.2.2 Base IBGE
Para realizar as análises que definimos, vamos precisar de poucas colunas:
- `municipio`: para conferir com os municípios da base de criminalidade
- `populacao_no_ultimo_censo_pessoas_2022`: para calcular os crimes a cada 100 mil pessoas
- `populacao_estimada_pessoas_2024`: para calcular, também, os crimes a cada 100 mil pessoas. Vamos calcular uma média

A base do IBGE é muito rica para tentar trazer hipóteses sobre criminalidade, mas nesse trabalho iremos desenvolver apenas esse índice.

A seguir vamos criar uma lista com as colunas relevantes.

In [0]:
# definindo as colunas e filtrando a base
colunas_relevantes_ibge = ['municipio', 'codigo', 'populacao_no_ultimo_censo__pessoas_2022', 'populacao_estimada__pessoas_2024']
df_ibge = df_ibge.select(*colunas_relevantes_ibge)

# checando se deu certo
df_ibge.printSchema()

### 3.3 Conversão de tipos
Vamos converter todas as colunas para os tipos corretos.

Na base de criminalidade temos duas colunas em formatos diferentes dos que precisamos, já a base do ibge está com os tipos corretos.

In [0]:
df_silver = df_silver \
    .withColumn("num_bo", col("num_bo").cast("int")) \
    .withColumn("latitude", col("latitude").cast("double"))

df_silver.printSchema()

### 3.4 Checando os nulos

#### 3.4.1 Base Criminalidade
Primeiro vamos visualizar os nulos de uma amostra de 100 mil dados (ao invés dos 3,6 milhões), em seguida vamos checar os valores absolutos.

In [0]:
# definindo amostra representativa com 100 mil valores
df_amostra = df_silver.sample(False, 100000 / df_silver.count(), seed=42).toPandas()

# gerando heatmap de nulos
plt.figure(figsize=(20, 6))
sns.heatmap(df_amostra.isnull().T, cbar=False, xticklabels=False, cmap="viridis")
plt.title("Heatmap de valores nulos (colunas no eixo Y)")
plt.ylabel("Colunas")
plt.xlabel("Registros (amostra de 100 mil)")
plt.show()

Observando a amostra, podemos chegar a algumas conclusões interessantes:
- Não há dados faltantes nos campos `nome_municipio_circunscricao` e `natureza_apurada` que são os campos mais importantes

In [0]:
# observando os valores absolutos de nulos
# Total de registros
total_registros = df_silver.count()

# Cálculo de nulos por coluna
resumo_nulos = df_silver.select([
    count(when(col(c).isNull(), c)).alias(c) for c in df_silver.columns
]).toPandas().T

# Formatando o DataFrame
resumo_nulos.columns = ["total_nulos"]
resumo_nulos["total_registros"] = total_registros
resumo_nulos["percentual_nulos"] = (resumo_nulos["total_nulos"] / total_registros * 100).round(2)

# Renomear índice para coluna
resumo_nulos = resumo_nulos.reset_index().rename(columns={"index": "coluna"})
resumo_nulos

Como realizaremos análises relacionadas, principalmente, à natureza do crime e localidade ao longo do tempo, vamos remover as entradas que estiverem com dados faltando nas colunas `natureza_apurada`, `ano_estatistica`, `mes_estatistica`, `nome_municipio_circunscricao`.

In [0]:
# removendo os nulos das colunas especificadas
colunas_filtrar = [
    "natureza_apurada",
    "mes_estatistica",
    "ano_estatistica",
    "nome_municipio_circunscricao"
]

df_silver = df_silver.dropna(subset=colunas_filtrar)


In [0]:
# observando os valores absolutos de nulos
# Total de registros
total_registros = df_silver.count()

# Cálculo de nulos por coluna
resumo_nulos = df_silver.select([
    count(when(col(c).isNull(), c)).alias(c) for c in df_silver.columns
]).toPandas().T

# Formatando o DataFrame
resumo_nulos.columns = ["total_nulos"]
resumo_nulos["total_registros"] = total_registros
resumo_nulos["percentual_nulos"] = (resumo_nulos["total_nulos"] / total_registros * 100).round(2)

# Renomear índice para coluna
resumo_nulos = resumo_nulos.reset_index().rename(columns={"index": "coluna"})
resumo_nulos

Como tínhamos a intenção de utilizar `num_bo` como chave primária (e ela está com muitos nulos), vamos remover essa coluna e criar outra chamada `id_bo` que cria um número único para cada boletim de ocorrência aberto.

#### 3.4.2 Base IBGE
Vamos conferir se existem dados faltantes.

In [0]:
# transformando em uma df pandas
df_ibge_pd = df_ibge.toPandas()

# gerando heatmap de nulos
plt.figure(figsize=(20, 6))
sns.heatmap(df_ibge_pd.isnull().T, cbar=False, xticklabels=False, cmap="viridis")
plt.title("Heatmap de valores nulos (colunas no eixo Y)")
plt.ylabel("Colunas")
plt.xlabel("Registros base IBGE")
plt.show()

Observando os dados, vemos que tem uma quantidade relevante de nulos, vamos observar caso a caso.

In [0]:
# Filtra todas as linhas que têm pelo menos um campo nulo
nulos_ibge = df_ibge_pd[df_ibge_pd.isnull().any(axis=1)]

# Visualiza os registros nulos
nulos_ibge

Como podemos ver, nenhum desses registros é, de fato, um registro. São textos no final da base. Vamos remover todos.

In [0]:
# removendo todos os nulos da base principal
df_ibge = df_ibge.dropna()

### 3.5 Base Criminalidade - Removendo `num_bo` e criando `id_bo`
Como iremos criar uma chave única, vamos adicionar algumas informações para facilitar o rastreamento do dado, caso a gente precise. O código ficará no formato `AAAAMMCCCRRRR`
- `AAAA`: Ano completo, como 2024
- `MM`: Mês em dois dígitos, como 03
- `CCC`: Três primeiros dígitos do nome do município, como SAO
- `RRRR`: 4 dígitos randômicos

In [0]:
# removendo num_bo
df_silver = df_silver.drop("num_bo")

In [0]:
# criando um id para os registros
# Gerar prefixos com ano, mês (com zero à esquerda) e 3 letras do município sem acento
df_silver = df_silver.withColumn("ano_str", col("ano_estatistica").cast("string")) \
    .withColumn("mes_str", lpad(col("mes_estatistica").cast("string"), 2, "0")) \
    .withColumn("municipio_prefixo", upper(substring(regexp_replace("nome_municipio_circunscricao", "[^a-zA-Z]", ""), 1, 3)))

# Criar sufixo aleatório a partir do ID interno (não totalmente aleatório, mas garante unicidade)
df_silver = df_silver.withColumn("sufixo", lpad((monotonically_increasing_id() % 10000).cast("int").cast("string"), 4, "0"))

# Concatenar tudo no formato final
df_silver = df_silver.withColumn("id_bo", concat_ws("", "ano_str", "mes_str", "municipio_prefixo", "sufixo"))

# Remover colunas temporárias
df_silver = df_silver.drop("ano_str", "mes_str", "municipio_prefixo", "sufixo")

# Contar valores únicos na coluna id_bo
df_silver.select("id_bo").distinct().count()

# Mostrar 10 registros incluindo todas as colunas
df_silver.show(10, truncate=False)

### 3.6 Ajustando as colunas strings

#### 3.6.1 Base Criminalidade
Vamos checar os valores de texto das colunas `nome_municipio_circunscricao` e `natureza_apurada` para entender se precisam de tratamento específico de texto.

In [0]:
# definir função
def normalizar_coluna_temporaria(df, coluna):
    return df.withColumn(f"{coluna}_clean",
                         lower(trim(regexp_replace(coluna, "[^a-zA-Z0-9çãáéêíóõú ]", ""))))

In [0]:
# definindo função para comparar as variáveis
def comparar_variantes(df, coluna):
    df_temp = normalizar_coluna_temporaria(df, coluna)

    print(f"\nColuna original: {coluna}")
    print("Valores únicos originais:")
    df.select(coluna).distinct().orderBy(coluna).show(20, truncate=False)

    print("\nApós normalização (sugestão de padronização):")
    df_temp.select(f"{coluna}_clean").distinct().orderBy(f"{coluna}_clean").show(20, truncate=False)


In [0]:
# rodar para as colunas de texto
colunas_texto = ["nome_municipio_circunscricao", "natureza_apurada"]

for col_texto in colunas_texto:
    comparar_variantes(df_silver, col_texto)

Como pudemos ver, as colunas possuem muitas inconsistências que precisam de ajuste. Vamos começar com alguns ajustes básicos em todas as colunas:
- Tornar todas as letras minúsculas
- Remover os acentos
- Remover os caracteres especiais
- Retirar espaços inúteis

In [0]:
# Função Python para remover acento e caracteres especiais
def normalizar_texto(texto):
    if texto is None:
        return None
    texto = texto.strip().lower()
    texto = unicodedata.normalize('NFKD', texto)
    texto = ''.join([c for c in texto if not unicodedata.combining(c)])  # remove acento
    texto = ''.join([c for c in texto if c.isalnum() or c == ' '])  # remove símbolos, mantém letras/números/espaço
    return texto

normalizar_udf = udf(normalizar_texto, StringType())


In [0]:
# Aplicar UDF nas colunas desejadas
df_silver = (
    df_silver
    .withColumn("municipio_clean", normalizar_udf(trim(col("nome_municipio_circunscricao"))))
    .withColumn("natureza_apurada_clean", normalizar_udf(trim(col("natureza_apurada"))))
)

Agora vamos checar os resultados, para ver se tem mudanças específicas para cada coluna.

In [0]:
# Cria o dataframe com as contagens
contagens_df = df_silver.select(
    countDistinct("nome_municipio_circunscricao").alias("municipio_original"),
    countDistinct("municipio_clean").alias("municipio_padronizado"),
    countDistinct("natureza_apurada").alias("natureza_original"),
    countDistinct("natureza_apurada_clean").alias("natureza_padronizada"),
)

# Transforma em pandas e transpõe
contagens_df.toPandas().transpose().rename(columns={0: "quantidade_distinta"})

Depois desses ajustes, vamos observar coluna por coluna e substituir os valores necessários.

#### 3.6.2 Nome do Município - Base Criminalidade e Base IBGE
Vamos checar para ver se esses nomes estão padronizados

In [0]:
df_silver.groupBy("municipio_clean") \
    .count() \
    .orderBy("count", ascending=True) \
    .show(100, truncate=False)

In [0]:
df_silver.select(countDistinct("municipio_clean").alias("total_municipios_distintos")).show()

O estado de São Paulo possui 645 municípios, vamos comparar essa lista com a do IBGE. Para isso precisamos tratar os nomes da base IBGE também.

In [0]:
# função pra remover acento e limpar o nome
def normalizar_municipio(texto):
    if texto is None:
        return None
    texto = texto.strip().lower()
    texto = unicodedata.normalize('NFKD', texto)
    texto = ''.join([c for c in texto if not unicodedata.combining(c)])
    texto = ''.join([c for c in texto if c.isalnum() or c == ' '])  # remove símbolos, mantém letras/números/espaço
    return texto

# UDF para aplicar no Spark
normalizar_udf = udf(normalizar_municipio, StringType())

# Criar a nova coluna limpa
df_ibge = df_ibge.withColumn("municipio_ibge_clean", normalizar_udf(trim(col("municipio"))))

In [0]:
df_ibge.printSchema()

In [0]:
# Função de normalização
def normalizar_municipio(texto):
    if texto is None:
        return None
    texto = texto.strip().lower()
    texto = unicodedata.normalize('NFKD', texto)
    texto = ''.join([c for c in texto if not unicodedata.combining(c)])
    texto = ''.join([c for c in texto if c.isalnum() or c == ' '])  # remove símbolos
    return texto

# UDF
normalizar_udf = udf(normalizar_municipio, StringType())

In [0]:
df_ibge = df_ibge.withColumn("municipio_ibge_clean", normalizar_udf(trim(col("municipio"))))

In [0]:
# Extrair os municípios únicos da sua base de criminalidade
df_municipios_crime = df_silver.select("municipio_clean").distinct()

# Identificar os que NÃO batem com o IBGE
df_municipios_nao_batem = df_municipios_crime.join(
    df_ibge,
    df_municipios_crime["municipio_clean"] == df_ibge["municipio_ibge_clean"],
    how="left_anti"
)

# Mostrar os esquisitos
df_municipios_nao_batem.orderBy("municipio_clean").show(100, truncate=False)

In [0]:
# Municípios únicos de cada base
df_municipios_crime = df_silver.select("municipio_clean").distinct()
df_municipios_ibge = df_ibge.select("municipio_ibge_clean").distinct()

In [0]:
# Os que estão na base de criminalidade mas NÃO estão no IBGE
df_somente_crime = df_municipios_crime.join(
    df_municipios_ibge,
    df_municipios_crime["municipio_clean"] == df_municipios_ibge["municipio_ibge_clean"],
    how="left_anti"
).withColumnRenamed("municipio_clean", "na_base_criminalidade")

# Os que estão no IBGE mas NÃO estão na base de criminalidade
df_somente_ibge = df_municipios_ibge.join(
    df_municipios_crime,
    df_municipios_ibge["municipio_ibge_clean"] == df_municipios_crime["municipio_clean"],
    how="left_anti"
).withColumnRenamed("municipio_ibge_clean", "na_base_ibge")

In [0]:
# Converter para pandas
pandas_crime = df_somente_crime.toPandas()
pandas_ibge = df_somente_ibge.toPandas()

# Juntar lado a lado com base no índice
comparacao_municipios = pandas_crime.merge(
    pandas_ibge,
    left_index=True,
    right_index=True,
    how="outer"
)

# Exibir (pode usar display(comparacao_municipios) se quiser no Databricks)
comparacao_municipios

In [0]:
# Dicionário de substituições
substituicoes_parciais = {
    "sjoao": "sao joao",
    "sjose": "sao jose",
    "sjoaquim": "sao joaquim",
    "sfrancisco": "sao francisco",
    "sisabel": "santa isabel",
    "sadelia": "santa adelia",
    "smercedes": "santa mercedes",
    "sroque": "sao roque",
    "sanastacio": "santo anastacio",
    "scarlos": "sao carlos",
    "sernestina": "santa ernestina",
    "smiguel": "sao miguel",
    "santonio": "santo antonio",
    "smaria": "santa maria",
    "ssimao": "sao simao",
    "slucia": "santa lucia",
    "sfe": "santa fe",
    "sluiz": "sao luiz",
    "ssebastiao": "sao sebastiao",
    "sbranca": "santa branca",
    "sexpedito": "santo expedito",
    "scruz": "santa cruz",
    "pta": "paulista",
    "srosa": "santa rosa",
    "svicente": "sao vicente",
    "sbernardo": "sao bernardo",
    "sbarbara": "santa barbara",
    "salbertina": "santa albertina",
    "sgertrudes": "santa gertrudes",
    "slourenco": "sao lourenco",
    "smanuel": "sao manuel",
    "spaulo": "sao paulo",
    "sclara": "santa clara",
    "sandre": "santo andre",
    "spedro": "sao pedro",
    "srita": "santa rita",
    "ssalete": "santa salete",
    "sbento": "sao bento",
    "scaetano": "sao caetano",
    "mirante paranapanema": "mirante do paranapanema",
    "pirapora bom jesus": "pirapora do bom jesus",
    "d oeste": "doeste",
    "sao joao duas pontes": "sao joao das duas pontes",
    "santa rita passa quatro": "santa rita do passa quatro",
    "santo antonio de aracangua": "santo antonio do aracangua",
    "d alho":"dalho",
    "florinia": "florinea",
    "campina monte alegre": "campina do monte alegre",
    "ribeirao indios": "ribeirao dos indios",
    "sdo": "santo do",
    "sto": "santo do",
    "biritibamirim": "biritiba mirim"

}

In [0]:
 # criando a fórmula para a substituição
def substituir_fragmentos_regex(texto):
    if texto is None:
        return None
    texto = texto.lower().strip()
    
    # Aplica substituição parcial com regex (ex: \bsjose\b pega palavra isolada ou dentro de texto)
    for chave, valor in substituicoes_parciais.items():
        texto = re.sub(rf"\b{re.escape(chave)}\b", valor, texto)
    return texto

substituir_fragmentos_udf = udf(substituir_fragmentos_regex, StringType())

In [0]:
# realizando as substituições
df_silver = df_silver.withColumn(
    "municipio_final",
    substituir_fragmentos_udf(col("municipio_clean"))
)

In [0]:
# substituindo na coluna certa
df_municipios_crime_final = df_silver.select("municipio_final").distinct()

In [0]:
# checando os que ainda não batem
df_municipios_nao_batem_final = df_municipios_crime_final.join(
    df_ibge.select("municipio_ibge_clean"),
    df_municipios_crime_final["municipio_final"] == df_ibge["municipio_ibge_clean"],
    how="left_anti"
)

In [0]:
df_municipios_crime_final = df_silver.select("municipio_final").distinct()
df_municipios_ibge = df_ibge.select("municipio_ibge_clean").distinct()

In [0]:
df_somente_crime = df_municipios_crime_final.join(
    df_municipios_ibge,
    df_municipios_crime_final["municipio_final"] == df_municipios_ibge["municipio_ibge_clean"],
    how="left_anti"
).withColumnRenamed("municipio_final", "na_base_criminalidade")

In [0]:
df_somente_ibge = df_municipios_ibge.join(
    df_municipios_crime_final,
    df_municipios_ibge["municipio_ibge_clean"] == df_municipios_crime_final["municipio_final"],
    how="left_anti"
).withColumnRenamed("municipio_ibge_clean", "na_base_ibge")

In [0]:
pandas_crime = df_somente_crime.toPandas()
pandas_ibge = df_somente_ibge.toPandas()

# Junta pelos índices (não tem correspondência direta, então é só pra exibir lado a lado)
comparacao_municipios = pandas_crime.merge(
    pandas_ibge,
    left_index=True,
    right_index=True,
    how="outer"
)

In [0]:
# substituindo a coluna antiga pelo nome padronizado final
df_silver = df_silver.drop("municipio_clean") \
    .withColumnRenamed("municipio_final", "municipio_clean")

Como podemos ver, não faltam mais correspondências na base de criminalidade, o que sobra na base do ibge são textos adjacentes. Por isso vamos seguir com a análise.

#### 3.6.2 `natureza_apurada`
Vamos checar as categorias e padronizar os textos.

In [0]:
df_silver.groupBy("natureza_apurada_clean") \
    .count() \
    .orderBy("count", ascending=True) \
    .show(1000, truncate=False)

Como pudemos observar acima, os nomes estão bem padronizados. Com os nomes padronizados, podemos seguir para a próxima etapa de preparação antes da análise.

#### 3.7 Ajustando as colunas de população - Base IBGE
Para criar o índice de criminalidade a cada 100 mil habitantes, vamos precisar da população de cada município, para isso precisamos ajustar as colunas relacionadas. O título delas está meio ruim, vamos ajustar agora também.

In [0]:
df_ibge.printSchema()

In [0]:
# ajustando o título das colunas relacionadas à população
novas_colunas_ibge = {
    "populacao_no_ultimo_censo__pessoas_2022": "pop_censo_22",
    "populacao_estimada__pessoas_2024": "pop_estimada_24"
}

In [0]:
# aplicando o dicionário novo
for antigo, novo in novas_colunas_ibge.items():
    df_ibge = df_ibge.withColumnRenamed(antigo, novo)

df_ibge.printSchema()

Com os nomes ajustados, vamos checar as estatísticas das duas colunas e ver se faz sentido com dados populacionais externos.

In [0]:
# checando mínimo, máximo e total da população
df_ibge.select("pop_censo_22", "pop_estimada_24").describe().show()

In [0]:
# vendo o total do estado
df_ibge.select(
    sum("pop_censo_22").alias("total_pop_22"),
    sum("pop_estimada_24").alias("total_pop_24")
).show()

De acordo com pesquisas externas, esses valores são compatíveis com a realidade. Com isso, vamos seguir com a análise.

### 4. Criação de novas colunas
A partir dos dados brutos tratados, vamos desenvolver novas colunas para suportar as análises que iremos realizar.

#### 4.1 Macro Categorias
Agora vamos criar a coluna `macro_categoria` para unir algumas categorias similares para simplificar a nossa análise, essas serão as categorias utilizadas, por serem:
- `estupro`: União entre `estupro` e `estupro de vulnerável`
- `roubo`: União entre `roubo a banco`, `roubo de carga`, `roubo de veiculo`, `roubo outros` e `latrocinio`
- `furto`: União entre `furto de carga`, `furto de veiculo`, `furto outros`
- `homicidio doloso`: União entre `homicidio doloso por acidente de transito`, `homicidio doloso`, `tentativa de homicidio`
- `homicidio culposo`: União entre `homicidio culposo outros`, `homicidio culposo por acidente de transito`
- `lesao corporal`: União entre `lesao corporal culposa por acidente de transito`, `lesao corporal culposa outras`, `lesao corporal seguida de morte`, `lesao corporal dolosa`
- `entorpecentes`: União entre `apreensao de entorpecentes`, `porte de entorpecentes`, `trafico de entorpecentes`

Algumas macro categorias não se associam especificamente, elas vão entrar como `tipos singulares`:
- `extorsao mediante sequestro`
- `porte de arma`


In [0]:

df_silver = df_silver.withColumn(
    "natureza_apurada_clean",
    regexp_replace(trim(regexp_replace(col("natureza_apurada_clean"), "\s+", " ")), "\s+$", "")
)

In [0]:
df_silver = df_silver.withColumn(
    "macro_categorias",
    when(col("natureza_apurada_clean").isin(
        "estupro", "estupro de vulneravel"), "estupro")

    .when(col("natureza_apurada_clean").isin(
        "roubo a banco", "roubo de carga", "roubo de veiculo", "roubo outros", "latrocinio"), "roubo")

    .when(col("natureza_apurada_clean").isin(
        "furto de carga", "furto de veiculo", "furto outros"), "furto")

    .when(col("natureza_apurada_clean").isin(
        "homicidio doloso", "homicidio doloso por acidente de transito", "tentativa de homicidio"), "homicidio doloso")

    .when(col("natureza_apurada_clean").isin(
        "homicidio culposo outros", "homicidio culposo por acidente de transito"), "homicidio culposo")

    .when(col("natureza_apurada_clean").isin(
        "lesao corporal culposa por acidente de transito", 
        "lesao corporal culposa outras", 
        "lesao corporal seguida de morte", 
        "lesao corporal dolosa"), "lesao corporal")

    .when(col("natureza_apurada_clean").isin(
        "apreensao de entorpecentes", 
        "porte de entorpecentes", 
        "trafico de entorpecentes"), "entorpecentes")

    .otherwise("tipos singulares")
)

Agora que criamos as macro categorias, vamos ver como a informação se agrupa nelas, através de um gráfico de barras.

In [0]:
# Agrupar e converter para pandas
df_plot = df_silver.groupBy("macro_categorias").count().toPandas()

# Ordenar do mais frequente pro menos
df_plot = df_plot.sort_values(by="count", ascending=False)

# Plotando
plt.figure(figsize=(12,6))
sns.barplot(x="count", y="macro_categorias", data=df_plot, palette="crest")
plt.title("Distribuição de Ocorrências por Macro Categoria", fontsize=14)
plt.xlabel("Número de Ocorrências")
plt.ylabel("Macro Categoria")
plt.tight_layout()
plt.show()

A distribuição faz sentido, e iremos utilizar essa categoria nas análises que iremos realizar.

#### 4.2 Tipos de Crime

Além das categorias criadas para este trabalho, vamos utilizar as categorias de tipos de crime de acordo com o código penal para enriquecer a análise de distribuição dos registros.

As categorias são:
- **contra a vida**: Engloba crimes que atentam diretamente contra a integridade física ou a existência da pessoa, como homicídios e lesões corporais.

- **contra o patrimônio**: Reúne crimes que envolvem a subtração ou destruição de bens alheios, incluindo furtos, roubos, extorsões e variantes como o latrocínio.

- **contra a dignidade sexual**: Refere-se a crimes que violam a liberdade e integridade sexual do indivíduo, como estupro e estupro de vulnerável.

- **contra a saúde pública**: Agrupa condutas que afetam o bem-estar coletivo, especialmente relacionadas ao tráfico, porte ou apreensão de drogas ilícitas.

- **contra a liberdade**: Envolve crimes que restringem a capacidade de ir e vir ou de autodeterminação da vítima, como o sequestro ou a extorsão mediante sequestro.

- **contra a segurança pública**: Inclui crimes que colocam em risco a ordem e o controle estatal, como o porte ilegal de arma de fogo.

- **não classificado**: Naturezas criminais que não se enquadram diretamente nas divisões do Código Penal ou demandam análise mais específica.

E a classificação das naturezas fica assim:


| natureza_apurada_clean                         | tipo_de_crime                       |
|------------------------------------------------|-------------------------------------|
| estupro                                        | Crimes contra a dignidade sexual    |
| estupro de vulneravel                          | Crimes contra a dignidade sexual    |
| roubo a banco                                  | Crimes contra o patrimônio          |
| roubo de carga                                 | Crimes contra o patrimônio          |
| roubo de veiculo                               | Crimes contra o patrimônio          |
| roubo  outros                                  | Crimes contra o patrimônio          |
| latrocinio                                     | Crimes contra o patrimônio          |
| furto de carga                                 | Crimes contra o patrimônio          |
| furto de veiculo                               | Crimes contra o patrimônio          |
| furto  outros                                  | Crimes contra o patrimônio          |
| homicidio doloso                               | Crimes contra a vida                |
| homicidio doloso por acidente de transito      | Crimes contra a vida                |
| tentativa de homicidio                         | Crimes contra a vida                |
| homicidio culposo outros                       | Crimes contra a vida                |
| homicidio culposo por acidente de transito     | Crimes contra a vida                |
| lesao corporal dolosa                          | Crimes contra a pessoa              |
| lesao corporal culposa por acidente de transito| Crimes contra a pessoa              |
| lesao corporal culposa  outras                 | Crimes contra a pessoa              |
| lesao corporal seguida de morte                | Crimes contra a pessoa              |
| apreensao de entorpecentes                     | Crimes contra a saúde pública       |
| porte de entorpecentes                         | Crimes contra a saúde pública       |
| trafico de entorpecentes                       | Crimes contra a saúde pública       |
| extorsao mediante sequestro                    | Crimes contra o patrimônio          |
| porte de arma                                  | Crimes contra a incolumidade pública|

In [0]:
# classificacao para a nova coluna de "tipo de crime"
tipo_crime_dict = {
    "estupro": "Crimes contra a dignidade sexual",
    "estupro de vulneravel": "Crimes contra a dignidade sexual",

    "roubo a banco": "Crimes contra o patrimônio",
    "roubo de carga": "Crimes contra o patrimônio",
    "roubo de veiculo": "Crimes contra o patrimônio",
    "roubo outros": "Crimes contra o patrimônio",
    "latrocinio": "Crimes contra o patrimônio",
    "furto de carga": "Crimes contra o patrimônio",
    "furto de veiculo": "Crimes contra o patrimônio",
    "furto outros": "Crimes contra o patrimônio",
    "extorsao mediante sequestro": "Crimes contra o patrimônio",

    "homicidio doloso": "Crimes contra a vida",
    "homicidio doloso por acidente de transito": "Crimes contra a vida",
    "tentativa de homicidio": "Crimes contra a vida",
    "homicidio culposo outros": "Crimes contra a vida",
    "homicidio culposo por acidente de transito": "Crimes contra a vida",

    "lesao corporal dolosa": "Crimes contra a pessoa",
    "lesao corporal culposa por acidente de transito": "Crimes contra a pessoa",
    "lesao corporal culposa outras": "Crimes contra a pessoa",
    "lesao corporal seguida de morte": "Crimes contra a pessoa",

    "apreensao de entorpecentes": "Crimes contra a saúde pública",
    "porte de entorpecentes": "Crimes contra a saúde pública",
    "trafico de entorpecentes": "Crimes contra a saúde pública",

    "porte de arma": "Crimes contra a incolumidade pública"
}

def classificar_tipo_crime(natureza):
    if natureza is None:
        return None
    return tipo_crime_dict.get(natureza.strip().lower(), "não classificado")

tipo_crime_udf = udf(classificar_tipo_crime, StringType())

df_silver = df_silver.withColumn("tipo_de_crime", tipo_crime_udf(col("natureza_apurada_clean")))


In [0]:
# gerando um gráfico para dar uma olhada
df_plot = df_silver.groupBy("tipo_de_crime").count().toPandas()
df_plot = df_plot.sort_values("count", ascending=True)

plt.figure(figsize=(10,6))
sns.barplot(x="count", y="tipo_de_crime", data=df_plot, palette="flare")

plt.title("Distribuição de Ocorrências por Tipo de Crime", fontsize=14, weight="bold")
plt.xlabel("Número de Ocorrências")
plt.ylabel("Tipo de Crime")
plt.grid(axis='x', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

Com as categorias de acordo com o código penal definidas, vamos para o próximo passo.

#### 5. Ajustando as colunas finais
Aqui vamos remover as colunas que não estão na sua versão final e alterar o nome de todas para o notebook Gold.

In [0]:
# definindo todas as colunas que vamos remover
remover_antigas = ['nome_municipio_circunscricao', 'natureza_apurada']
remover_antigas_ibge = ['municipio']

# removendo as colunas
df_silver = df_silver.drop(*remover_antigas)
df_ibge = df_ibge.drop(*remover_antigas_ibge)

In [0]:
# definindo o nome das novas colunas
novos_nomes = {
    'ano_estatistica': 'ano',
    'mes_estatistica': 'mes',
    'natureza_apurada_clean': 'natureza',
    'municipio_clean': 'municipio',
    'macro_categorias': 'categoria',
    'tipo_de_crime': 'tipo_crime'

}

novo_ibge = {
    'municipio_ibge_clean': 'municipio',
    'codigo': 'codigo_ibge'
}

In [0]:
# aplicando o dicionário novo
for antigo, novo in novos_nomes.items():
    df_silver = df_silver.withColumnRenamed(antigo, novo)

df_silver.printSchema()

In [0]:
# aplicando o dicionário novo
for antigo, novo in novo_ibge.items():
    df_ibge = df_ibge.withColumnRenamed(antigo, novo)

df_ibge.printSchema()

Agora vamos exportar os dados limpos, particionados por ano, para facilitar a rodar o notebook Gold com mais rapidez.

In [0]:
dbutils.fs.rm("/FileStore/mvp_criminalidade_sp/silver", True)

In [0]:
# definindo o caminho dos arquivos
caminho_silver = "/FileStore/mvp_criminalidade_sp/silver"
caminho_silver_ibge = "/FileStore/mvp_criminalidade_sp/silver/ibge"

# salvando os arquivos nas pastas corretas
df_silver.write.mode("overwrite").partitionBy("ano").parquet(caminho_silver)
df_ibge.write.mode("overwrite").parquet(caminho_silver_ibge)

# checando se deu certo
display(dbutils.fs.ls("/FileStore/mvp_criminalidade_sp/silver"))

In [0]:
# exportando os csvs para colocar no GitHub
anos = [2022, 2023, 2024]

for ano in anos:
    df_silver.filter(col("ano") == ano) \
        .coalesce(1) \
        .write \
        .mode("overwrite") \
        .option("header", True) \
        .csv(f"/FileStore/mvp_criminalidade_sp/export/silver_crimes_csv/ano={ano}")

In [0]:
df_ibge.coalesce(1) \
    .write \
    .mode("overwrite") \
    .option("header", True) \
    .csv("/FileStore/mvp_criminalidade_sp/export/silver_ibge_csv")

In [0]:
# baixando os arquivos para colocar no github - ibge
arquivos = dbutils.fs.ls("/FileStore/mvp_criminalidade_sp/export/silver_ibge_csv")
for arquivo in arquivos:
    if arquivo.name.endswith(".csv"):
        print(f"Link de download:\nhttps://community.cloud.databricks.com/files/mvp_criminalidade_sp/export/silver_ibge_csv/{arquivo.name}")

In [0]:
# baixando os arquivos para colocar no github - ibge
anos = [2022, 2023, 2024]

for ano in anos:
    arquivos = dbutils.fs.ls(f"/FileStore/mvp_criminalidade_sp/export/silver_crimes_csv/ano={ano}")
    for arquivo in arquivos:
        if arquivo.name.endswith(".csv"):
            print(f"Ano {ano} - Link de download:\nhttps://community.cloud.databricks.com/files/mvp_criminalidade_sp/export/silver_crimes_csv/ano={ano}/{arquivo.name}\n")
